Hello!

The goal of this colab is to build an understanding of what machine learning is, stripped down to a problem where a human (you!) could accomplish the same task given enough brainpower and time.

In this colab you will create a model to classify handwritten digits (0-9) from the classic [MNIST dataset](https://en.wikipedia.org/wiki/MNIST_database). 

**Human learning:** First the tuning of the model will be done by you. Using a handful of image features describing the dominant line angles in each image, you will try to find the best weighting for each feature to separate the digit classes.

**Machine learning:** Next we will let a machine do the same task, faster better stronger, but still the same task.  

In both settings -- human learning and machine learning -- we used a machine to help with the computation by speeding through thousands of simple calculations.  The difference between the two settings, and the real power of machine learning, is from when the machine also does the tuning itself.

In [ ]:
#@title python imports
!pip install panel --upgrade
!pip install ipywidgets_bokeh
import numpy as np
import matplotlib.pyplot as plt
# %matplotlib inline
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn import svm
from IPython.display import clear_output
from skimage.transform import hough_line
from matplotlib.figure import Figure
import matplotlib.gridspec as gridspec

import ipywidgets as widgets
from IPython.display import display

import panel as pn
pn.extension('ipywidgets')


---
---


**Data and model parameters**

Decide which digits to try to classify.  We recommend starting with just 0 and 1, to keep it easy.  

The second parameter is the number of line angles to detect in each image.  With 4 line angles (the default), there will be 4 values for every image, which we will call "features".  The first value will be the presence of 45 degree lines in the image (detected by a Hough transform if you're interested).  The second feature value will measure the presence of vertical lines, then 135 degrees, and finally horizontal lines.  More line angles to detect mean more values to describe each image.

We also include a feature that counts the number of "on" pixels in each image, represented by an uppercase Σ (sigma, for summation).

In [ ]:
digits_to_use = "0,1" #@param {type:"string"}
number_line_angles =  4#@param {type:"integer"}

digits_to_use = sorted([int(val) for val in digits_to_use.strip().split(',')]) # a hacky way to get a list of ints from an input form


---
---


We load the dataset and do some preprocessing of the images.  We'll only use a thousand images for the training set.

**If you change either of the parameters above, you'll need to re-run the following cell**

In [ ]:
#@title Load the digit data and process into features
def preprocess(example):
  return tf.image.convert_image_dtype(tf.squeeze(example['image']), tf.float32), example['label']

def filter_digits(image, label):
  return tf.math.reduce_any(tf.equal(label, tf.constant(digits_to_use, dtype=tf.int64)))

dataset_train = tfds.load('mnist', split='train').map(preprocess).filter(filter_digits)
dataset_test = tfds.load('mnist', split='test').map(preprocess).filter(filter_digits)

number_train_images = 1024
number_test_images = 128
train_images, train_labels = next(iter(dataset_train.batch(number_train_images)))
test_images, test_labels = next(iter(dataset_test.batch(number_test_images)))

_, image_height, image_width = train_images.shape

tested_line_angles = np.linspace(-np.pi / 2, np.pi / 2, number_line_angles, endpoint=False)
train_features = np.concatenate([np.sum(train_images, axis=(1, 2)).reshape([-1, 1]),
                          np.stack([np.max(hough_line(train_images[i].numpy(), theta=tested_line_angles)[0], axis=0) for i in range(number_train_images)], 0)
                     ], -1)

test_features = np.concatenate([np.sum(test_images, axis=(1, 2)).reshape([-1, 1]),
                          np.stack([np.max(hough_line(test_images[i].numpy(), theta=tested_line_angles)[0], axis=0) for i in range(number_test_images)], 0)
                     ], -1)

train_features = (train_features - np.min(train_features, axis=0, keepdims=True)) / (np.max(train_features, axis=0, keepdims=True) - np.min(train_features, axis=0, keepdims=True))
test_features = (test_features - np.min(test_features, axis=0, keepdims=True)) / (np.max(test_features, axis=0, keepdims=True) - np.min(test_features, axis=0, keepdims=True))

---
---

**The model is simple:** for each image, take its feature values and multiply by a set of weights that indicate if it is a `0`.  Then multiply by another set of weights that indicate if it is a `1`, and so on, until you have a value for each possible digit.  The digit with the largest value is the one you classify it as.  

The code below shows a randomly selected image from each possible class, along with that image's feature values and the classification output (as probabilities) for the current model. 

Below this are the weights for each feature, for each digit.  Tune them so that they are high when the feature is usually high (for example, the vertical line feature in `1`'s is usually pretty strong, so we'd increase that weight), and low when the feature is usually low.  The most important features are those that vary a lot between the classes: look for the strongest effect there.

In [ ]:
#@title Tune a model to classify the digits!
color_brewer_colors = ['#a6cee3', '#1f78b4', '#b2df8a', '#33a02c', '#fb9a99', '#e31a1c', '#fdbf6f', '#ff7f00', '#cab2d6', '#6a3d9a']  ## ty https://colorbrewer2.org/
number_features = train_features.shape[1]
if number_line_angles == 4:
  feature_labels = ['Σ', '--', '\\', '|',  '/']
  feature_colors = color_brewer_colors[1::2]
elif number_features<=10:
  feature_labels = ['Σ'] + [f'{np.rad2deg(rad):.1f}' for rad in tested_line_angles]
  feature_colors = color_brewer_colors[:number_features]
else:
  feature_labels = ['']*number_features
  feature_colors = ['#e41a1c']*number_features

def display_rand_images():
  inches_per_subplot = 2.5
  number_features = train_features.shape[-1]
  num_classes = len(digits_to_use)
  image_inds = [np.random.choice(np.where(train_labels==digit)[0]) for digit in digits_to_use]

  class_weights = []
  for digit_ind, digit in enumerate(digits_to_use):
    feature_weights = [slider.value for slider in sliders[digit_ind][::-1]]
    class_weights.append(np.sum(np.reshape(feature_weights, [1, -1])*train_features, axis=-1))  ## Shape [N], one scalar per image
  class_weights = np.stack(class_weights, -1)  ## Shape [N, num_digits]
  # Simplify the fitting situation by solving for biases that even out the class predictions (so that 1/num_classes are positive and the rest neg for each class weighting)
  biases = np.percentile(class_weights, 100*(1-1./len(digits_to_use)), axis=0)
  class_weights -= biases.reshape([1, -1])
  assigned_probs = tf.nn.softmax(class_weights, axis=1)
  acc = np.mean(np.int32(digits_to_use)[np.argmax(assigned_probs, axis=1)] == train_labels)

  fig = Figure(figsize=(3*inches_per_subplot*1.4, inches_per_subplot*len(digits_to_use)))
  specs = gridspec.GridSpec(ncols=3, nrows=len(digits_to_use), figure=fig)
  for plot_ind, image_ind in enumerate(image_inds):
    ax = fig.add_subplot(specs[plot_ind, 0])
    ax.imshow(train_images[image_ind], cmap='gray_r')
    ax.axis('off')

    ax = fig.add_subplot(specs[plot_ind, 1])
    ax.barh(np.arange(number_features), train_features[image_ind], color=feature_colors)
    ax.set_xlim(0, 1)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.set_yticks(range(number_features))
    ax.set_yticklabels(feature_labels, fontsize=16)
    ax.set_xticks([])
    if not plot_ind:
      ax.set_title('Image features', fontsize=14)

    ax = fig.add_subplot(specs[plot_ind, 2])

    ax.bar(np.arange(num_classes), assigned_probs[image_ind], color='#798086')
    ax.set_ylim(0, 1)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.set_xticks(range(num_classes))
    ax.set_xticklabels(digits_to_use, fontsize=16)
    ax.set_yticks([0, 0.5, 1.])
    if not plot_ind:
      ax.set_title('Assigned probability', fontsize=14)
  return fig

def train_svm(_):
  svm_instance = svm.LinearSVC(max_iter=10_000)
  svm_instance.fit(train_features, train_labels)
  coeffs = svm_instance.coef_
  ## If there are only 2 classes, then there's only 1 weight vector; just duplicate it (negated) for the second class
  if len(digits_to_use) == 2:
    for slider_ind, slider in enumerate(sliders[0]):
      slider.value = -coeffs[0, len(sliders[0])-1-slider_ind]
    for slider_ind, slider in enumerate(sliders[1]):
      slider.value = coeffs[0, len(sliders[1])-1-slider_ind]
  else:
    for digit_ind, digit in enumerate(digits_to_use):
      for slider_ind, slider in enumerate(sliders[digit_ind]):
        slider.value = coeffs[digit_ind, len(sliders[digit_ind])-1-slider_ind]
  return

def refresh_images(_):
  mpl_pane.object = display_rand_images()

def evaluate_model(_):
  class_weights = []
  for digit_ind, digit in enumerate(digits_to_use):
    feature_weights = [slider.value for slider in sliders[digit_ind][::-1]]
    class_weights.append(np.sum(np.reshape(feature_weights, [1, -1])*train_features, axis=-1))  ## Shape [N], one scalar per image
  class_weights = np.stack(class_weights, -1)  ## Shape [N, num_digits]
  # Simplify the fitting situation by solving for biases that even out the class predictions (so that 1/num_classes are positive and the rest neg for each class weighting)
  biases = np.percentile(class_weights, 100*(1-1./len(digits_to_use)), axis=0)
  class_weights -= biases.reshape([1, -1])
  assigned_probs = tf.nn.softmax(class_weights, axis=-1)
  acc = np.mean(np.int32(digits_to_use)[np.argmax(assigned_probs, axis=1)] == train_labels)
  accuracy_text.value = f'Accuracy: {acc:.4f}'
  return


evaluate_button = pn.widgets.Button(name="Evaluate model", button_type='primary')
refresh_button = pn.widgets.Button(name="Refresh images", button_type='success')
accuracy_text = pn.widgets.TextInput(value='')
evaluate_bundle = pn.Row(evaluate_button, accuracy_text)
ml_button = pn.widgets.Button(name="MACHINE LEARNING", button_type='danger')

sliders = [[pn.widgets.FloatSlider(name=feature_labels[feature_ind], value=0, start=-5, end=5, step=0.05, bar_color=feature_colors[feature_ind]) for feature_ind in range(number_features)[::-1]] for digit in digits_to_use]
mpl_pane = pn.pane.Matplotlib(display_rand_images(), dpi=72, tight=True)

evaluate_button.on_click(evaluate_model)
ml_button.on_click(train_svm)
refresh_button.on_click(refresh_images)
items = []
for thing in zip(digits_to_use, [pn.Column(*slider_bunch, background='WhiteSmoke', width=200) for slider_bunch in sliders]):
  items.append([f'WEIGHTS FOR {thing[0]}'] + thing[1])
evaluate_model('')

display(mpl_pane)
display(refresh_button)

grid = pn.GridBox(*items, ncols=len(digits_to_use))
display(grid)
display(evaluate_bundle)
display(ml_button)

---
---

Finally, since machine learning can sift through tons of features without caring what they mean, we can even hand in completely random features.  It turns out [random projection](https://en.wikipedia.org/wiki/Random_projection) can be a pretty powerful method of dimensionality reduction.  How well does it do to classify digits?  You may want to re-sample the features a few times, and see how much the performance changes.

**How much more difficult is it to tell what's been learned?**

In [ ]:
digits_to_use = "0,1,2,3" #@param {type:"string"}
number_random_features =  40#@param {type:"integer"}

digits_to_use = sorted([int(val) for val in digits_to_use.strip().split(',')]) # a hacky way to get a list of ints from an input form

In [ ]:
#@title Reprocess into features (and resample the random projections)
dataset_train = tfds.load('mnist', split='train').map(preprocess).filter(filter_digits)
dataset_test = tfds.load('mnist', split='test').map(preprocess).filter(filter_digits)

train_images, train_labels = next(iter(dataset_train.batch(number_train_images)))
test_images, test_labels = next(iter(dataset_test.batch(number_test_images)))

_, image_height, image_width = train_images.shape

random_vectors = np.random.normal(size=(number_random_features, 28*28))
train_features = np.dot(np.reshape(train_images, [-1, 28*28]), random_vectors.T)
test_features = np.dot(np.reshape(test_images, [-1, 28*28]), random_vectors.T)

train_features = (train_features - np.min(train_features, axis=0, keepdims=True)) / (np.max(train_features, axis=0, keepdims=True) - np.min(train_features, axis=0, keepdims=True))
test_features = (test_features - np.min(test_features, axis=0, keepdims=True)) / (np.max(test_features, axis=0, keepdims=True) - np.min(test_features, axis=0, keepdims=True))

In [ ]:
#@title Random feature model!
color_brewer_colors = ['#a6cee3', '#1f78b4', '#b2df8a', '#33a02c', '#fb9a99', '#e31a1c', '#fdbf6f', '#ff7f00', '#cab2d6', '#6a3d9a']  ## ty https://colorbrewer2.org/

number_features = train_features.shape[-1]
if number_features <= 10:
  feature_colors = color_brewer_colors[:number_features]
else:
  feature_colors = ['#e41a1c']*number_features
feature_labels = ['']*number_features

def display_rand_images():
  inches_per_subplot = 2.5
  num_classes = len(digits_to_use)
  image_inds = [np.random.choice(np.where(train_labels==digit)[0]) for digit in digits_to_use]

  class_weights = []
  for digit_ind, digit in enumerate(digits_to_use):
    feature_weights = [slider.value for slider in sliders[digit_ind][::-1]]
    class_weights.append(np.sum(np.reshape(feature_weights, [1, -1])*train_features, axis=-1))  ## Shape [N], one scalar per image
  class_weights = np.stack(class_weights, -1)  ## Shape [N, num_digits]
  # Simplify the fitting situation by solving for biases that even out the class predictions (so that 1/num_classes are positive and the rest neg for each class weighting)
  biases = np.percentile(class_weights, 100*(1-1./len(digits_to_use)), axis=0)
  class_weights -= biases.reshape([1, -1])
  assigned_probs = tf.nn.softmax(class_weights, axis=1)
  acc = np.mean(np.int32(digits_to_use)[np.argmax(assigned_probs, axis=1)] == train_labels)

  fig = Figure(figsize=(3*inches_per_subplot*1.4, inches_per_subplot*len(digits_to_use)))
  specs = gridspec.GridSpec(ncols=3, nrows=len(digits_to_use), figure=fig)
  for plot_ind, image_ind in enumerate(image_inds):
    ax = fig.add_subplot(specs[plot_ind, 0])
    ax.imshow(train_images[image_ind], cmap='gray_r')
    ax.axis('off')

    ax = fig.add_subplot(specs[plot_ind, 1])
    ax.barh(np.arange(number_features), train_features[image_ind], color=feature_colors)
    ax.set_xlim(0, 1)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.set_yticks(range(number_features))
    ax.set_yticklabels(feature_labels, fontsize=16)
    ax.set_xticks([])
    if not plot_ind:
      ax.set_title('Image features', fontsize=14)

    ax = fig.add_subplot(specs[plot_ind, 2])

    ax.bar(np.arange(num_classes), assigned_probs[image_ind], color='#798086')
    ax.set_ylim(0, 1)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.set_xticks(range(num_classes))
    ax.set_xticklabels(digits_to_use, fontsize=16)
    ax.set_yticks([0, 0.5, 1.])
    if not plot_ind:
      ax.set_title('Assigned probability', fontsize=14)
  return fig

def train_svm(_):
  svm_instance = svm.LinearSVC(max_iter=10_000)
  svm_instance.fit(train_features, train_labels)
  coeffs = svm_instance.coef_
  ## If there are only 2 classes, then there's only 1 weight vector; just duplicate it (negated) for the second class
  if len(digits_to_use) == 2:
    for slider_ind, slider in enumerate(sliders[0]):
      slider.value = -coeffs[0, len(sliders[0])-1-slider_ind]
    for slider_ind, slider in enumerate(sliders[1]):
      slider.value = coeffs[0, len(sliders[1])-1-slider_ind]
  else:
    for digit_ind, digit in enumerate(digits_to_use):
      for slider_ind, slider in enumerate(sliders[digit_ind]):
        slider.value = coeffs[digit_ind, len(sliders[digit_ind])-1-slider_ind]
  return

def refresh_images(_):
  mpl_pane.object = display_rand_images()

def evaluate_model(_):
  class_weights = []
  for digit_ind, digit in enumerate(digits_to_use):
    feature_weights = [slider.value for slider in sliders[digit_ind][::-1]]
    class_weights.append(np.sum(np.reshape(feature_weights, [1, -1])*train_features, axis=-1))  ## Shape [N], one scalar per image
  class_weights = np.stack(class_weights, -1)  ## Shape [N, num_digits]
  # Simplify the fitting situation by solving for biases that even out the class predictions (so that 1/num_classes are positive and the rest neg for each class weighting)
  biases = np.percentile(class_weights, 100*(1-1./len(digits_to_use)), axis=0)
  class_weights -= biases.reshape([1, -1])
  assigned_probs = tf.nn.softmax(class_weights, axis=-1)
  acc = np.mean(np.int32(digits_to_use)[np.argmax(assigned_probs, axis=1)] == train_labels)
  accuracy_text.value = f'Accuracy: {acc:.4f}'
  return


evaluate_button = pn.widgets.Button(name="Evaluate model", button_type='primary')
refresh_button = pn.widgets.Button(name="Refresh images", button_type='success')
accuracy_text = pn.widgets.TextInput(value='')
evaluate_bundle = pn.Row(evaluate_button, accuracy_text)
ml_button = pn.widgets.Button(name="MACHINE LEARNING", button_type='danger')

sliders = [[pn.widgets.FloatSlider(name=feature_labels[feature_ind], value=0, start=-5, end=5, step=0.05, bar_color=feature_colors[feature_ind]) for feature_ind in range(number_features)[::-1]] for digit in digits_to_use]
mpl_pane = pn.pane.Matplotlib(display_rand_images(), dpi=72, tight=True)

evaluate_button.on_click(evaluate_model)
ml_button.on_click(train_svm)
refresh_button.on_click(refresh_images)
items = []
for thing in zip(digits_to_use, [pn.Column(*slider_bunch, background='WhiteSmoke', width=200) for slider_bunch in sliders]):
  items.append([f'WEIGHTS FOR {thing[0]}'] + thing[1])
evaluate_model('')

display(mpl_pane)
display(refresh_button)

grid = pn.GridBox(*items, ncols=len(digits_to_use))
display(grid)
display(evaluate_bundle)
display(ml_button)

When interpretability is difficult, it's important to have other checks that the model is performing as well as you think it is.  If you tried a bunch of random projections in the cells above, you might have found some that work particularly well with the specific set of training images, but wouldn't work well on unseen images.  This is the problem of **overfitting**.

Evaluate the accuracy on an unseen test set below, to see if it matches the accuracy on the training set.

In [ ]:
#@title Evaluate on the test set
class_weights = []
for digit_ind, digit in enumerate(digits_to_use):
  feature_weights = [slider.value for slider in sliders[digit_ind][::-1]]
  class_weights.append(np.sum(np.reshape(feature_weights, [1, -1])*test_features, axis=-1))  ## Shape [N], one scalar per image
class_weights = np.stack(class_weights, -1)  ## Shape [N, num_digits]
# Simplify the fitting situation by solving for biases that even out the class predictions (so that 1/num_classes are positive and the rest neg for each class weighting)
biases = np.percentile(class_weights, 100*(1-1./len(digits_to_use)), axis=0)
class_weights -= biases.reshape([1, -1])
assigned_probs = tf.nn.softmax(class_weights, axis=-1)
acc = np.mean(np.int32(digits_to_use)[np.argmax(assigned_probs, axis=1)] == test_labels)
print(f'Test set accuracy: {acc:.4f}')